# Préparation environnement de travail, importations des csv et fusion des bases

In [2]:
# monter le google drive pour qu'il soit accessible ("Fichiers" dans colab - liste à gauche)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Importation des modules nécessaires

import pandas as pd
import numpy as np

# Option à modifier pour visualiser la totalité des colonnes

pd.set_option('display.max_columns', None)

In [2]:
df=pd.read_pickle('/content/drive/MyDrive/Projet Pompier/Données complémentaires/data_modif2.pkl')

In [3]:
df.drop(['TurnoutTimeSeconds','TravelTimeSeconds','FalseAlarm','DelayCode_Description','TimeOnIncidentSeconds','TimeTotalSeconds','PropertyType','IncidentNumber','DateAndTimeMobilised','DateAndTimeMobile','DateAndTimeMobile','DateAndTimeArrived','DateAndTimeLeft','DeployedFromStation_Code','DateOfCall','TimeOfCall','CalYear_y','TimeOfCall','HourOfCall_y'],1,inplace=True)

In [4]:
df=df[df['PumpOrder']==1]

In [5]:
conditions = [
    (df['PumpOrder']==1) & (df['AttendanceTimeSeconds']>340),
    (df['PumpOrder']==1) & (df['AttendanceTimeSeconds']<=340)]
choices = [1,0]
df['Goal'] = np.select(conditions, choices)

In [6]:
##conditions = [
#    (df['PumpOrder']==1) & (df['AttendanceTimeSeconds']>340),
#    (df['PumpOrder']==1) & (df['AttendanceTimeSeconds']<=340),
#    (df['PumpOrder']==2) & (df['AttendanceTimeSeconds']>460),
#    (df['PumpOrder']==2) & (df['AttendanceTimeSeconds']<=460),]
#choices = [1,0,1,0]
#df['Goal'] = np.select(conditions, choices)

In [7]:
df[['AttendanceTimeSeconds','PumpOrder','Goal']].head()

,AttendanceTimeSeconds,PumpOrder,Goal
0,348.0,1,1
1,254.0,1,0
2,212.0,1,0
5,340.0,1,0
6,775.0,1,1


In [8]:
df.drop('PumpOrder',1,inplace=True)

In [9]:
df=df.select_dtypes(exclude='object').join(pd.get_dummies(df.select_dtypes('object')))

In [10]:
#dummy1 = pd.get_dummies(df['PumpOrder'], prefix='PumpOrder')
#df = pd.concat([df, dummy1], axis = 1)
#df.drop('PumpOrder',1,inplace=True)

In [11]:
dummy2 = pd.get_dummies(df['CalYear_x'], prefix='Year')
df = pd.concat([df, dummy2], axis = 1)
df.drop('CalYear_x',1,inplace=True)

In [12]:
for i in ['NumStationsWithPumpsAttending','NumPumpsAttending','Outdoor Structure','Road Vehicle','Outdoor','Dwelling','Other Residential','Aircraft','Boat','Rail Vehicle','Non Residential']:
    dummy3 = pd.get_dummies(df[i],prefix=i)
    df = pd.concat([df, dummy3], axis = 1)
df.drop(['NumStationsWithPumpsAttending','NumPumpsAttending','Outdoor Structure','Road Vehicle','Outdoor','Dwelling','Other Residential','Aircraft','Boat','Rail Vehicle','Non Residential'],1,inplace=True)

In [13]:
df.head()

,HourOfCall_x,AttendanceTimeSeconds,Domestic Incidents,Fire,Local Emergencies,Major Environmental Disasters,Prior Arrangement,Use of Special Operations Room,Distance,Goal,Resource_Code_A211,Resource_Code_A212,Resource_Code_A231,Resource_Code_A241,Resource_Code_A242,Resource_Code_A281,Resource_Code_A301,Resource_Code_A311,Resource_Code_A321,Resource_Code_A322,Resource_Code_A331,Resource_Code_A332,Resource_Code_A341,Resource_Code_A342,Resource_Code_A351,Resource_Code_A352,Resource_Code_A361,Resource_Code_A371,Resource_Code_A381,Resource_Code_A391,Resource_Code_A392,Resource_Code_A401,Resource_Code_A402,Resource_Code_A411,Resource_Code_A412,Resource_Code_A431,Resource_Code_A432,Resource_Code_E211,Resource_Code_E221,Resource_Code_E231,Resource_Code_E232,Resource_Code_E251,Resource_Code_E252,Resource_Code_E271,Resource_Code_E281,Resource_Code_E291,Resource_Code_E301,Resource_Code_E311,Resource_Code_E341,Resource_Code_E351,Resource_Code_E361,Resource_Code_E371,Resource_Code_E381,Resource_Code_E391,Resource_Code_E392,Resource_Code_E401,Resource_Code_E402,Resource_Code_E411,Resource_Code_E412,Resource_Code_E421,Resource_Code_E431,Resource_Code_F211,Resource_Code_F221,Resource_Code_F231,Resource_Code_F241,Resource_Code_F251,Resource_Code_F261,Resource_Code_F281,Resource_Code_F282,Resource_Code_F291,Resource_Code_F301,Resource_Code_F321,Resource_Code_F322,Resource_Code_F331,Resource_Code_F341,Resource_Code_F351,Resource_Code_F361,Resource_Code_F362,Resource_Code_F371,Resource_Code_F381,Resource_Code_F391,Resource_Code_F401,Resource_Code_F411,Resource_Code_F412,Resource_Code_F421,Resource_Code_F422,Resource_Code_F431,Resource_Code_F432,Resource_Code_F441,Resource_Code_F451,Resource_Code_F571,Resource_Code_G211,Resource_Code_G221,Resource_Code_G222,Resource_Code_G231,Resource_Code_G232,Resource_Code_G241,Resource_Code_G242,Resource_Code_G251,Resource_Code_G261,Resource_Code_G271,Resource_Code_G272,Resource_Code_G281,Resource_Code_G291,Resource_Code_G301,Resource_Code_G302,Resource_Code_G311,Resource_Code_G312,Resource_Code_G321,Resource_Code_G331,Resource_Code_G341,Resource_Code_G351,Resource_Code_G361,Resource_Code_G362,Resource_Code_G371,Resource_Code_G381,Resource_Code_G382,Resource_Code_G391,Resource_Code_G392,Resource_Code_G401,Resource_Code_G561,Resource_Code_H211,Resource_Code_H221,Resource_Code_H222,Resource_Code_H241,Resource_Code_H242,Resource_Code_H251,Resource_Code_H252,Resource_Code_H261,Resource_Code_H262,Resource_Code_H271,Resource_Code_H281,Resource_Code_H291,Resource_Code_H311,Resource_Code_H312,Resource_Code_H321,Resource_Code_H322,Resource_Code_H331,Resource_Code_H341,Resource_Code_H351,Resource_Code_H352,Resource_Code_H361,Resource_Code_H371,Resource_Code_H381,Resource_Code_H382,Resource_Code_H391,Resource_Code_H401,Resource_Code_H411,Resource_Code_H412,Resource_Code_H421,Resource_Code_H431,Resource_Code_H432,DeployedFromStation_Name_Acton,DeployedFromStation_Name_Addington,DeployedFromStation_Name_Barking,DeployedFromStation_Name_Barnet,DeployedFromStation_Name_Battersea,DeployedFromStation_Name_Beckenham,DeployedFromStation_Name_Bethnal Green,DeployedFromStation_Name_Bexley,DeployedFromStation_Name_Biggin Hill,DeployedFromStation_Name_Brixton,DeployedFromStation_Name_Bromley,DeployedFromStation_Name_Chelsea,DeployedFromStation_Name_Chingford,DeployedFromStation_Name_Chiswick,DeployedFromStation_Name_Clapham,DeployedFromStation_Name_Croydon,DeployedFromStation_Name_Dagenham,DeployedFromStation_Name_Deptford,DeployedFromStation_Name_Dockhead,DeployedFromStation_Name_Dowgate,DeployedFromStation_Name_Ealing,DeployedFromStation_Name_East Greenwich,DeployedFromStation_Name_East Ham,DeployedFromStation_Name_Edmonton,DeployedFromStation_Name_Eltham,DeployedFromStation_Name_Enfield,DeployedFromStation_Name_Erith,DeployedFromStation_Name_Euston,DeployedFromStation_Name_Feltham,DeployedFromStation_Name_Finchley,DeployedFromStation_Name_Forest Hill,DeployedFromStation_Name_Fulham,DeployedFromStation_Name_Greenwich,DeployedFromSt

In [14]:
df.drop('AttendanceTimeSeconds',1,inplace=True)

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
df[['HourOfCall_x','Distance']]=sc.fit_transform(df[['HourOfCall_x','Distance']])  

In [16]:
data=df.drop('Goal',1)
target=df['Goal']

##from sklearn.feature_selection import SelectKBest,f_classif
##selector=SelectKBest(f_classif,k=45)
##selector.fit_transform(data,target)
##data_kbest=data[data.columns[selector.get_support()]]

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression().fit(X_train,y_train)

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
sg=SGDClassifier(class_weight='balanced',random_state=42)
print(cross_val_score(sg,X_train,y_train,cv=5,scoring='recall'))

[0.69945494 0.78095869 0.76658176 0.75287825 0.73530311]


In [23]:
print(cross_val_score(sg,X_train,y_train,cv=5,scoring='recall').mean())

0.7470353498356446


In [25]:
sg.fit(X_train,y_train)

SGDClassifier(class_weight='balanced', random_state=42)

In [26]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_train,sg.predict(X_train)))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.83      0.70      0.84      0.76      0.59    202521
          1       0.66      0.70      0.83      0.68      0.76      0.57     98152

avg / total       0.79      0.79      0.74      0.79      0.76      0.58    300673



In [27]:
y_pred=sg.predict(X_test)

In [28]:
pd.crosstab(y_test,y_pred)

col_0,0,1
Goal,,
0,41758,8679
1,7302,17430


In [29]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.83      0.70      0.84      0.76      0.59     50437
          1       0.67      0.70      0.83      0.69      0.76      0.58     24732

avg / total       0.79      0.79      0.75      0.79      0.76      0.59     75169



In [37]:
#1er model relativement bon sans overfitting

In [30]:
tt=pd.DataFrame(sg.coef_.T)
tt.rename(columns={0:'coeff'},inplace=True)
tt['name']=X_train.columns

In [33]:
tt.sort_values(by='coeff',ascending=False,key=abs).head(10)

,coeff,name
225,-8.772680,DeployedFromStation_Name_Ruislip
106,-7.722587,Resource_Code_G321
7,2.117905,Distance
351,-1.532368,IncidentStationGround_Ruislip
427,1.001842,NumPumpsAttending_1
312,-0.618779,IncidentStationGround_Harrow
339,-0.602956,IncidentStationGround_Northolt
272,0.564930,AddressQualifier_On motorway / elevated road
186,0.563202,DeployedFromStation_Name_Harrow
417,-0.551344,NumStationsWithPumpsAttending_1


In [18]:
!pip install tpot

  Using cached TPOT-0.11.7-py3-none-any.whl (87 kB)
  Using cached stopit-1.1.2-py3-none-any.whl
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)
  Using cached deap-1.3.1-cp38-cp38-win_amd64.whl (108 kB)


You should consider upgrading via the 'c:\users\ludo\anaconda3\python.exe -m pip install --upgrade pip' command.


In [ ]:
from tpot import TPOTClassifier
tpot=TPOTClassifier(generations=5,population_size=50,n_jobs=-1,scoring='recall',verbosity=2,random_state=42)
tpot.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]

In [22]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
clb=BernoulliNB(SGDClassifier(alpha=0.01, eta0=0.01, fit_intercept=False, l1_ratio=0.5, 
                          learning_rate=invscaling, loss=perceptron, penalty=elasticnet, power_t=0.1), alpha=1.0, fit_prior=False)

NameError: ignored

In [ ]:
df.to_pickle('/content/drive/MyDrive/Projet Pompier/Données complémentaires/data_modif3.pkl')

In [ ]:
df=pd.read_pickle('/content/drive/MyDrive/Projet Pompier/Données complémentaires/data_modif3.pkl')

In [ ]:
!pip install optuna

In [42]:
import optuna
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split


def objective(trial):
    # hyperparameter setting
    loss = trial.suggest_categorical('loss',('hinge','log','modified_huber','squared_hinge'))
    penalty= trial.suggest_categorical('penalty',('l1','l2','elasticnet'))
    alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
    if penalty == 'elasticnet':
        l1_ratio=trial.suggest_uniform('l1_ratio',0.1,0.95)
        model=SGDClassifier(loss=loss,penalty=penalty,alpha=alpha,l1_ratio=l1_ratio,class_weight='balanced')
    else:
        model=SGDClassifier(loss=loss,penalty=penalty,alpha=alpha,class_weight='balanced')
     
    
    # data loading and train-test split
    data=df.drop('Goal',1)
    target=df['Goal']
    X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
    # model training and evaluation
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = sklearn.metrics.recall_score(y_test, y_pred)
 
    # output: evaluation score
    return score

study = optuna.create_study(direction='maximize',pruner=optuna.pruners.MedianPruner(n_startup_trials=5,
                                                               n_warmup_steps=30,
                                                               interval_steps=10))
study.optimize(objective, n_trials=30)

[I 2022-02-02 20:16:52,391] A new study created in memory with name: no-name-12a149a1-4117-42b2-b59b-c7d2a7ff05e5
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

[I 2022-02-02 20:17:02,102] Trial 0 finished with value: 0.0 and parameters: {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.6964384477273877, 'l1_ratio': 0.7927948586427427}. Best is trial 0 with value: 0.0.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

[I 2022-02-02 20:33:20,098] Trial 1 finished with value: 0.62

In [44]:
print('recall: ' + str(study.best_value))
print('Best parameter: ' + str(study.best_params))

recall: 0.9996765324276241
Best parameter: {'loss': 'hinge', 'penalty': 'l2', 'alpha': 1.252793374351399}


In [45]:
optuna.visualization.plot_optimization_history(study)

In [46]:
sg=SGDClassifier(class_weight='balanced',random_state=42)
sg.set_params(**study.best_params)
sg.fit(X_train, y_train)

SGDClassifier(alpha=1.252793374351399, class_weight='balanced', random_state=42)

In [47]:
from sklearn.metrics import classification_report
y_pred=sg.predict(X_test)
print(pd.crosstab(y_test,y_pred))
print(classification_report(y_test,y_pred))

col_0   0      1
Goal            
0      12  50425
1       0  24732
              precision    recall  f1-score   support

           0       1.00      0.00      0.00     50437
           1       0.33      1.00      0.50     24732

    accuracy                           0.33     75169
   macro avg       0.66      0.50      0.25     75169
weighted avg       0.78      0.33      0.16     75169



In [ ]:
df.to_pickle('/content/drive/MyDrive/Projet Pompier/Données complémentaires/data_modif4.pkl')

NameError: ignored

In [ ]:
df=pd.read_pickle('/content/drive/MyDrive/Projet Pompier/Données complémentaires/data_modif4.pkl')

In [ ]:
target.value_counts()

0    252958
1    122884
Name: Goal, dtype: int64

In [ ]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective1(trial):
    data=df.drop('Goal',1)
    target=df['Goal']
    X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        'learning_rate':trial.suggest_float("learning_rate", 1e-8, 1.0, log=True)
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    recall = sklearn.metrics.recall_score(y_test, pred_labels,pos_label=1)
    return recall


study1 = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_startup_trials=5,
                                                               n_warmup_steps=30,
                                                               interval_steps=10), direction="maximize")
study1.optimize(objective1, n_trials=100)
print(study1.best_trial)

[I 2022-01-30 20:46:09,183] A new study created in memory with name: no-name-1374100a-bae1-4ccb-8e5e-55a5a72bd591
[W 2022-01-30 20:46:13,896] Trial 0 failed because of the following error: TypeError("fit() got an unexpected keyword argument 'evals'")
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-27-73c7f48760c7>", line 40, in objective1
    bst = XGBClassifier.fit(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
TypeError: fit() got an unexpected keyword argument 'evals'


TypeError: ignored

In [ ]:
print('recall: ' + str(study1.best_value))
print('Best parameter: ' + str(study1.best_params))

recall: 1.0
Best parameter: {'booster': 'gblinear', 'lambda': 2.6831669876751306e-05, 'alpha': 0.00031932221251140943, 'learning_rate': 1.3604943952384372e-08}


In [ ]:
print('fbeta: ' + str(study1.best_value))
print('Best parameter: ' + str(study1.best_params))

fbeta: 1.0
Best parameter: {'booster': 'gblinear', 'lambda': 0.6292822328241894, 'alpha': 3.214411706376571e-05, 'learning_rate': 0.0009688458101249592}


In [ ]:
params={'booster': 'gbtree', 'lambda': 0.09917839681080542, 'alpha': 0.0026686276468855475, 'learning_rate': 4.145039440178865e-07,
        'max_depth': 2, 'eta': 0.0012307086740043676, 'gamma': 0.001313071089969198, 'grow_policy': 'depthwise'}
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)
bst1 = xgb.train(params,dtrain,evals=[(dvalid, "validation")])
preds1 = bst1.predict(dvalid)
pred_labels1 = np.rint(preds1)

[0]	validation-rmse:0.5
[1]	validation-rmse:0.5
[2]	validation-rmse:0.5
[3]	validation-rmse:0.5
[4]	validation-rmse:0.5
[5]	validation-rmse:0.5
[6]	validation-rmse:0.5
[7]	validation-rmse:0.5
[8]	validation-rmse:0.5
[9]	validation-rmse:0.5


In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(pd.crosstab(y_test,pred_labels1))
print(classification_report_imbalanced(y_test,pred_labels1))

col_0    0.0    1.0
Goal               
0      47699   2738
1      14432  10300
                   pre       rec       spe        f1       geo       iba       sup

          0       0.77      0.95      0.42      0.85      0.63      0.41     50437
          1       0.79      0.42      0.95      0.55      0.63      0.37     24732

avg / total       0.78      0.77      0.59      0.75      0.63      0.40     75169



In [ ]:
params={'booster': 'gblinear', 'lambda': 2.6831669876751306e-05, 'alpha': 0.00031932221251140943, 'learning_rate': 1.3604943952384372e-08}
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)
bst1 = xgb.train(params,dtrain,evals=[(dvalid, "validation")])
preds1 = bst1.predict(dvalid)
pred_labels1 = np.rint(preds1)

[0]	validation-rmse:0.5
[1]	validation-rmse:0.5
[2]	validation-rmse:0.5
[3]	validation-rmse:0.5
[4]	validation-rmse:0.5
[5]	validation-rmse:0.5
[6]	validation-rmse:0.5
[7]	validation-rmse:0.5
[8]	validation-rmse:0.5
[9]	validation-rmse:0.5


In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(pd.crosstab(y_test,pred_labels1))
print(classification_report_imbalanced(y_test,pred_labels1))

col_0    0.0  1.0
Goal             
0      50320  117
1      24452  280
                   pre       rec       spe        f1       geo       iba       sup

          0       0.67      1.00      0.01      0.80      0.11      0.01     50437
          1       0.71      0.01      1.00      0.02      0.11      0.01     24732

avg / total       0.68      0.67      0.34      0.55      0.11      0.01     75169



In [ ]:
import xgboost as xgb
params={'booster': 'dart', 'lambda': 0.020443518288511875, 'alpha': 1.1269077123126336e-05, 'learning_rate': 3.020805690724844e-08, 'max_depth': 8, 'eta': 0.000150583630316509, 'gamma': 1.493906460795342e-06, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 3.281030666354044e-07, 'skip_drop': 6.855232468359635e-05}
X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)
bst2 = xgb.train(params,dtrain,evals=[(dvalid, "validation")])
preds2 = bst2.predict(dvalid)
pred_labels2 = np.rint(preds2)

[0]	validation-rmse:0.5
[1]	validation-rmse:0.5
[2]	validation-rmse:0.5
[3]	validation-rmse:0.5
[4]	validation-rmse:0.5
[5]	validation-rmse:0.5
[6]	validation-rmse:0.5
[7]	validation-rmse:0.5
[8]	validation-rmse:0.5
[9]	validation-rmse:0.5


In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(pd.crosstab(y_test,pred_labels2))
print(classification_report_imbalanced(y_test,pred_labels2))

col_0    0.0    1.0
Goal               
0      47145   3292
1      12864  11868
                   pre       rec       spe        f1       geo       iba       sup

          0       0.79      0.93      0.48      0.85      0.67      0.47     50437
          1       0.78      0.48      0.93      0.60      0.67      0.43     24732

avg / total       0.78      0.79      0.63      0.77      0.67      0.46     75169



In [ ]:
from imblearn.under_sampling import ClusterCentroids
undersample = ClusterCentroids()
X_train_cc, y_train_cc = undersample.fit_resample(X_train, y_train)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
sg=SGDClassifier(class_weight='balanced',random_state=42)
print(cross_val_score(sg,X_train,y_train,cv=5,scoring='recall'))
sg.fit(X_train,y_train)